In [0]:
!pip install pandas
!pip install -q tensorflow_text

In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text


pd.set_option('max_colwidth', 150)  # Increase column width
data = pd.read_excel("Chatbot_qa.xlsx", encoding='utf8')
data.head()

,Context,Answer,face masks,racism/xenophobia,outbreak origin,cases,dog,death,washing hands,vaccine/cure,SARS,cruise ships,prevention,america/US/president,lockdown/social distancing/quarantine
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.,0,0.0,0,0,1,0,0,0,0,0,0,0,0
1,What do coronaviruses do to humans?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases",0,0.0,0,0,0,0,0,0,0,0,0,0,0
2,What is the difference between COVID-19 and the coronavirus?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak...,0,1.0,1,1,0,0,0,0,0,0,0,0,0
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, so...",0,0.0,0,1,0,1,0,0,0,0,0,0,0
4,Does everyone experience symptoms if they're infected?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing...,0,0.0,0,1,0,1,0,0,0,0,0,0,0


In [0]:
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
# Create response embeddings
# response_encodings = module.signatures['response_encoder'](
#         input=tf.constant(data.Answer),
#         context=tf.constant(data.Context))['outputs']

In [0]:
from googlesearch import search

def get_top_questions(qa_array, N, filtered_data):
  top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
  sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]
  top_possible_questions = pd.Series.to_numpy(filtered_data.Context[sorted_top_response_indicies])
  top_possible_questions = np.insert(top_possible_questions, N, 'None of the above questions are similar to what I asked')
  
  frame = {'Which is your question?': top_possible_questions}
  questions_df = pd.DataFrame(frame, index=np.arange(1, N+2, 1))
  return questions_df

def get_response(x):
  # TODO:
  # Do I/O here
  # return numbers as ints, no/yes as "no" or "yes"
  if (x == 0): return 'no'
  if (x == 1): return 6

def get_label_from_question(question):
  # TODO: get label from question
  return 'dog'

def get_question():
  # TODO:
  # get question from I/O
  # return a string
  return "Can my pet dog give me covid-19?"

def is_exit(question):
  if (question == 'bye'):
    return True
  return False

def get_google_search(query):
  search_result_list = list(search(query, tld="com", num=10, stop=3, pause=1))
  return search_result_list[0]

def convo(question):
  label = get_label_from_question(question)

  filtered_data = data.loc[data[label] == 1, ['Context', 'Answer']].reset_index(drop=True)
  response_encodings = module.signatures['response_encoder'](
        input=tf.constant(filtered_data.Answer),
        context=tf.constant(filtered_data.Context))['outputs']

  question_encodings = module.signatures['question_encoder'](
    tf.constant(question)
  )['outputs']

  print("********")
  print(question)
  print("********")

# Determine scores for question and answer encoding products
  qa_inner_prods = np.inner(question_encodings, response_encodings)
  qa_array = qa_inner_prods[0]
  print(data.Answer[np.argmax(qa_inner_prods, axis=1)])
  print("Did I answer your question?")
  response = get_response(0)

  print("********")
  print(response)
  print("********")

  if (response == 'no'):
    N = 5
    questions_df = get_top_questions(qa_array, N, filtered_data)



    print("Are any of the following the questions you asked?")
    questions_df.to_html('possible_questions.html')
    display(HTML('possible_questions.html'))
    print("Please type the number corresponding to the closest question you asked")
    response = get_response(1)

    print("********")
    print(response)
    print("********")

    
    top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
    sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]

    if (response >= 1 and response <= 5):
      print(data.Context[sorted_top_response_indicies[response-1]])
      print(data.Answer[sorted_top_response_indicies[response-1]])   
    else:
      print("Here's a link that hopefully answers your question")
      print("If this does not answer your question, please try rephrasing your question")
      print(get_google_search(question))
      return
  print("What would you like to know next about Covid-19? If you want to stop talking to me, simply type 'bye'")
  return

def main():
  print("Hi, what would you like to know about Covid-19?")
  
  count = 0

  while(True):
    if (count == 1): break
    question = get_question()
    if (is_exit(question)):
      break
    convo(question)
    count = count + 1
  print("Bye now!")

  
main()


Hi, what would you like to know about Covid-19?
********
Can my pet dog give me covid-19?
********
8    People can also catch COVID-19 if they breathe in droplets from a person with COVID-19 who coughs out or exhales droplets. This is why it is impor...
Name: Answer, dtype: object
Did I answer your question?
********
no
********
Are any of the following the questions you asked?


,Which is your question?
1,Can humans get coronavirus from dogs or cats?
2,Can I catch COVID-19 from my pet?
3,Can humans be infected with the COVID-19 from an animal source?
4,What is a coronavirus?
5,What do animals have to do with the coronavirus?
6,None of the above questions are similar to what I asked


Please type the number corresponding to the closest question you asked
********
6
********
Here's a link that hopefully answers your question
If this does not answer your question, please try rephrasing your question
https://www.oregonvma.org/care-health/zoonotic-diseases/coronavirus-faq
Bye now!


In [59]:
# Methods to put into flask

def respond_highest_scoring_answer(question, label):

  data = pd.read_excel("Chatbot_qa.xlsx", encoding='utf8')
  module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

  # will need to get the data as a dataframe beforehand or sometime here
  filtered_data = data.loc[data[label] == 1, ['Context', 'Answer']].reset_index(drop=True)
  response_encodings = module.signatures['response_encoder'](
        input=tf.constant(filtered_data.Answer),
        context=tf.constant(filtered_data.Context))['outputs']

  question_encodings = module.signatures['question_encoder'](
    tf.constant(question)
  )['outputs']
  qa_inner_prods = np.inner(question_encodings, response_encodings)
  qa_array = qa_inner_prods[0]
  return filtered_data.Answer[np.argmax(qa_inner_prods, axis=1)].values[0]

def show_top_5_questions(question, label):

    data = pd.read_excel("Chatbot_qa.xlsx", encoding='utf8')
    module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

    # will need to get the data as a dataframe beforehand or sometime here
    filtered_data = data.loc[data[label] == 1, ['Context', 'Answer']].reset_index(drop=True)
    response_encodings = module.signatures['response_encoder'](
          input=tf.constant(filtered_data.Answer),
          context=tf.constant(filtered_data.Context))['outputs']

    question_encodings = module.signatures['question_encoder'](
      tf.constant(question)
    )['outputs']
    qa_inner_prods = np.inner(question_encodings, response_encodings)
    qa_array = qa_inner_prods[0]

    N = 5

    top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
    sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]
    top_possible_questions = pd.Series.to_numpy(filtered_data.Context[sorted_top_response_indicies])
    top_possible_questions = np.insert(top_possible_questions, N, 'None of the above questions are similar to what I asked')
    
    frame = {'Which is your question?': top_possible_questions}
    questions_df = pd.DataFrame(frame, index=np.arange(1, N+2, 1))
    
    return '_'.join(questions_df['Which is your question?'].values)
    
def get_top_5_answer(question, label, response):

    data = pd.read_excel("Chatbot_qa.xlsx", encoding='utf8')
    module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

    # will need to get the data as a dataframe beforehand or sometime here
    filtered_data = data.loc[data[label] == 1, ['Context', 'Answer']].reset_index(drop=True)
    response_encodings = module.signatures['response_encoder'](
          input=tf.constant(filtered_data.Answer),
          context=tf.constant(filtered_data.Context))['outputs']

    question_encodings = module.signatures['question_encoder'](
      tf.constant(question)
    )['outputs']
    qa_inner_prods = np.inner(question_encodings, response_encodings)
    qa_array = qa_inner_prods[0]

    N = 5
    top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
    sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]
    top_possible_questions = pd.Series.to_numpy(filtered_data.Context[sorted_top_response_indicies])
    top_possible_questions = np.insert(top_possible_questions, N, 'None of the above questions are similar to what I asked')
    
    frame = {'Which is your question?': top_possible_questions}
    questions_df = pd.DataFrame(frame, index=np.arange(1, N+2, 1))
    
    return filtered_data.Answer[sorted_top_response_indicies[response-1]]



def get_google_search(question):
  search_result_list = list(search(question, tld="com", num=10, stop=3, pause=1))
  return search_result_list[0]

print(get_top_5_answer("Can my pet dog give me covid-19?", "dog", 5))


Can humans get coronavirus from dogs or cats?_Can I catch COVID-19 from my pet?_Can humans be infected with the COVID-19 from an animal source?_What is a coronavirus?_What do animals have to do with the coronavirus?_None of the above questions are similar to what I asked
